In [1]:
import wandb
import os
import json
import numpy as np
from utils.pareto import extreme_prune

In [2]:
api = wandb.Api(timeout=120)
baseline_runs = api.runs("wilrop/MORL-Baselines-v1")

In [3]:
num_evals = 100
for run in baseline_runs:
    algo = run.config['algo']
    env_id = run.config['env_id']
    seed = run.config['seed']
    eval_fronts = []
    print(f'Processing {env_id} - {algo} - {seed}')
    history_dicts = [h for h in run.scan_history(keys=['eval/front', 'global_step'])]
    
    if len(history_dicts) < num_evals:
        interval = 1
        offset = 0
    else:
        interval = len(history_dicts) // num_evals
        offset = len(history_dicts) % num_evals
        
    for i, h in enumerate(history_dicts):
        if (i + 1 - offset) % interval != 0:
            continue
        data_path = os.path.join('data', env_id, algo, str(seed))
        pf_path = h['eval/front']['path']
        combined_path = os.path.join(data_path, pf_path)
        run.file(pf_path).download(root=data_path)

        with open(combined_path, 'r') as f:
            front_data = json.load(f)
            front = np.array(front_data['data'])
            eval_fronts.append((h['global_step'], front))
    
    eval_fronts = sorted(eval_fronts, key=lambda x: x[0])
    
    for i, eval_front in eval_fronts:
        front_path = os.path.join('fronts', env_id, algo, str(seed), f'front_{i}.npy')
        os.makedirs(os.path.dirname(front_path), exist_ok=True)
        np.save(front_path, eval_front)
    
    merged_fronts = np.concatenate([front for _, front in eval_fronts], axis=0)
    final_front = extreme_prune(merged_fronts)
    final_front_path = os.path.join('fronts', env_id, algo, str(seed), 'final_front.npy')
    os.makedirs(os.path.dirname(final_front_path), exist_ok=True)
    np.save(final_front_path, final_front)

Processing deep-sea-treasure-concave-v0 - GPI-LS - 2
Processing deep-sea-treasure-concave-v0 - GPI-LS - 3
Processing deep-sea-treasure-concave-v0 - GPI-LS - 4
Processing deep-sea-treasure-concave-v0 - GPI-LS - 0
Processing deep-sea-treasure-concave-v0 - GPI-LS - 1
Processing deep-sea-treasure-concave-v0 - PCN - 4
Processing deep-sea-treasure-concave-v0 - PCN - 2
Processing deep-sea-treasure-concave-v0 - PCN - 1
Processing deep-sea-treasure-concave-v0 - PCN - 0
Processing deep-sea-treasure-concave-v0 - PCN - 3
Processing mo-reacher-v4 - PCN - 2
Processing minecart-v0 - PCN - 2
Processing minecart-v0 - PCN - 3
Processing minecart-v0 - PCN - 1
Processing minecart-v0 - PCN - 0
Processing mo-reacher-v4 - PCN - 0
Processing minecart-v0 - PCN - 4
Processing mo-reacher-v4 - PCN - 1
Processing mo-reacher-v4 - PCN - 4
Processing mo-reacher-v4 - PCN - 3
Processing deep-sea-treasure-concave-v0 - Envelope - 3
Processing deep-sea-treasure-concave-v0 - Envelope - 0
Processing deep-sea-treasure-concav